In [1]:
param_url = 'https://www.ebay.co.uk/sch/i.html?_dkr=1&iconV2Request=true&_blrs=recall_filtering&_ssn=spares-hut&store_cat=0&store_name=allautopartsonline&_oac=1&_nkw=mirror'

In [2]:
if param_url.startswith('https://www.ebay.com/'):
    market = 'US'
elif param_url.startswith('https://www.ebay.de/'):
    market = 'DE'
elif param_url.startswith('https://www.ebay.co.uk/'):
    market = 'UK'
elif param_url.startswith('https://www.ebay.com.au/'):
    market = 'AU'

market

'UK'

In [3]:
dict_param = {}
list_param = param_url.split('?')[1].split('&') if '?' in param_url else []
for param in list_param:
    dict_param[param.split('=')[0]] = param.split('=')[1]

dict_param

{'_dkr': '1',
 'iconV2Request': 'true',
 '_blrs': 'recall_filtering',
 '_ssn': 'spares-hut',
 'store_cat': '0',
 'store_name': 'allautopartsonline',
 '_oac': '1',
 '_nkw': 'mirror'}

In [4]:
dict_param.pop('_pgn', '')
dict_param['orig_cvip'] = 'true'
dict_param['_sop'] = '16'
dict_param['_ipg'] = '240'
dict_param['_udlo'] = ''
dict_param['_udhi'] = ''

if market == 'US':
    dict_param['_ul'] = 'US'
    dict_param['_stpos'] = '91710'
elif market == 'DE':
    dict_param['_ul'] = 'DE'
    dict_param['_stpos'] = '10115'
elif market == 'UK':
    dict_param['_ul'] = 'GB'
    dict_param['_stpos'] = 'BS81QU'
elif market == 'AU':
    dict_param['_ul'] = 'AU'
    dict_param['_stpos'] = '2019'

dict_param

{'_dkr': '1',
 'iconV2Request': 'true',
 '_blrs': 'recall_filtering',
 '_ssn': 'spares-hut',
 'store_cat': '0',
 'store_name': 'allautopartsonline',
 '_oac': '1',
 '_nkw': 'mirror',
 'orig_cvip': 'true',
 '_sop': '16',
 '_ipg': '240',
 '_udlo': '',
 '_udhi': '',
 '_ul': 'GB',
 '_stpos': 'BS81QU'}

In [5]:
param_url = param_url.split('?')[0] + '?'
for key, value in dict_param.items():
    param_url += f'{key}={value}&'

param_url

'https://www.ebay.co.uk/sch/i.html?_dkr=1&iconV2Request=true&_blrs=recall_filtering&_ssn=spares-hut&store_cat=0&store_name=allautopartsonline&_oac=1&_nkw=mirror&orig_cvip=true&_sop=16&_ipg=240&_udlo=&_udhi=&_ul=GB&_stpos=BS81QU&'

In [6]:
import requests
import json

import sys
sys.path.append('../../00.Tools')
from crawler_configuration import get_proxy

resp = requests.get(param_url[:-1],
                    # proxies=get_proxy(),
                    timeout=(10, 10)).text

resp

'<!DOCTYPE html><!--[if IE 9]><html class="ie9" lang="en"><![endif]--><!--[if gt IE 9]><!--><html lang="en"><!--<![endif]--><!--M#s0-2--><noscript class=x-page-config></noscript><!--M/--><head><meta http-equiv=X-UA-Compatible content=IE=edge><script>\n    (function () {\n        \'use strict\';\n        if (window.PerformanceObserver && performance && performance.mark && performance.getEntriesByName) {\n            window.SRP = window.SRP || {};\n            var paintObserver = new window.PerformanceObserver(function (list) {\n                var paintEntries = list.getEntries();\n                paintEntries.sort(function (a, b) {\n                    return a.startTime - b.startTime;\n                });\n                // begin looking for TTI at first contentful paint\n                if (!paintEntries || paintEntries.length < 2) {\n                    // to avoid undefined issue on Safari\n                    return;\n                }\n                var interactiveWindow = pai

In [7]:
from bs4 import BeautifulSoup
from lxml import etree

soup = BeautifulSoup(resp.strip(), 'lxml')
html = etree.HTML(str(soup))

with open('./page.html', 'w', encoding='UTF-8') as file:
    file.write(str(soup))

print(soup.prettify())

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [8]:
count = int(html.xpath('//h1[@class="srp-controls__count-heading"]/span[1]/text()')[0].strip().replace(',', '').replace('.', ''))

count

461

In [9]:
import math

page = math.ceil(count / 240) + 1

page

3

In [10]:
import pandas as pd

output = pd.DataFrame({'Market': market,
                       'Page': [_+1 for _ in range(page)],
                       'Page Url': [f'{param_url}_pgn={_+1}' for _ in range(page)]})

output.to_excel('./1.page.xlsx', index=False)

output

,Market,Page,Page Url
0,UK,1,https://www.ebay.co.uk/sch/i.html?_dkr=1&iconV...
1,UK,2,https://www.ebay.co.uk/sch/i.html?_dkr=1&iconV...
2,UK,3,https://www.ebay.co.uk/sch/i.html?_dkr=1&iconV...
